## Reading the files in wx_data folder to a Pandas dataframe

In [ ]:
import glob
import pandas as pd
import os

In [ ]:
data_files = glob.glob('wx_data/*.txt')

In [ ]:
print(data_files)

In [ ]:
## Reading all the text files to a dataframe 
df = pd.concat((pd.read_csv(file, sep='\t', header=None) for file in data_files))

In [ ]:
## Defining column names to the dataframe
df.columns = ['weather_date','max_temp','min_temp','precipitation']

In [ ]:
## Modifying the type for date column (weather_date)
df['weather_date'] = pd.to_datetime(df['weather_date'].astype(str), format='%Y%m%d')

#### Modifying the values in temperature and precipitation column to contain the right values in celsius and millimeters respectively

In [ ]:
df['max_temp'] = df['max_temp']/10

In [ ]:
df['min_temp'] = df['min_temp']/10
df['precipitation'] = df['precipitation']/10

In [ ]:
pd.set_option('display.max_rows',None)

#### Adding a column Updated_at to keep track of changes in the values in the table

In [ ]:
df['updated_at'] = pd.Timestamp.utcnow()

## Connecting to MySQL database using the SQLAlchemy module

In [ ]:
import mysql.connector as msql
from mysql.connector import Error

In [ ]:
from sqlalchemy import create_engine
my_conn = create_engine("mysql+pymysql://root:<password>@localhost/<database>")

In [ ]:
# Converting the files to a sql table named weather 
df.to_sql('weather', con=my_conn, if_exists = 'replace', index=False)

### In the similar way, when new data needs to be updated to the table, the data is first inserted to a 'Staging table'. This Staging table will only hold the most recent incoming data. The data is loaded in the same way as mentioned above to load the initial data.

In [ ]:
my_conn.execute('select * from stage_data').fetchall()

In [ ]:
# Using MySQL connector now to connect to the MySQL table

import mysql.connector

# Connect to the MySQL database
cnx = mysql.connector.connect(user='<username>', password='<password>',
                              host='localhost', database='CortevaCrop')
cursor = cnx.cursor()

In [ ]:
# Define the source table and destination table
source_table = 'stage_data'
destination_table = 'weather'


In [ ]:
# Define the columns to be merged
columns = ['weather_date', 'max_temp', 'min_temp', 'precipitation', 'updated_at']

# Define the query to merge the tables
merge_query = f"INSERT INTO {destination_table} SELECT * FROM {source_table} " \
              f"ON DUPLICATE KEY UPDATE " \
              f"{', '.join([f'{col}=VALUES({col})' for col in columns])}"

In [ ]:
# Execute the query to merge the tables
cursor.execute(merge_query)
cnx.commit()

# Close the database connection
cursor.close()
cnx.close()

### Creating a Table which will hold aggregated data. It is a dimension table for the 'weather' table. 

In [ ]:
# create a cursor object to execute SQL queries
cursor = cnx.cursor()

# create a new table to store the yearly weather statistics
create_table_query = """
    CREATE TABLE IF NOT EXISTS yearly_weather_stats (
        year INT NOT NULL,
        avg_max_temperature DEFAULT NULL,
        avg_min_temperature DEFAULT NULL,
        total_precipitation DEFAULT NULL,
        PRIMARY KEY (year)
    )
"""
cursor.execute(create_table_query)

# calculate the yearly average min and max temperature and total accumulated precipitation and insert the results into the new table
insert_query = """
    INSERT INTO yearly_weather_stats (year, avg_max_temperature,avg_min_temperature, total_precipitation)
    SELECT YEAR(weather_date) AS year,
           AVG(max_temp) AS avg_max_temperature,
           AVG(min_temp) AS avg_min_temperature,
           SUM(precipitation) AS total_precipitation
    FROM weather
    ON DUPLICATE KEY UPDATE year = year, 
                            avg_max_temperature = avg_max_temperature,
                            avg_max_temperature = avg_max_temperature,
                            total_precipitation = total_precipitation
    GROUP BY YEAR(weather_date)
"""
cursor.execute(insert_query)

# commit the changes and close the cursor and database connection
cnx.commit()
cursor.close()
cnx.close()